In [58]:
import pandas as pd
import json
import ast

In [59]:
#we have association rule in csv format which will be available in filesystem(local or s3).
def csvToPandasParser(filename):
    ARDataFrame = pd.read_csv(filename, sep = ',', header='infer', warn_bad_lines=True)
    return ARDataFrame

In [60]:
#Input Tokis are processed. It will be changed when request handling logic is added but currently we are taking it from a file.
def inputHandler(filename):
    newTokis = pd.read_csv(filename, sep = ',', header='infer', warn_bad_lines=True)
    return newTokis

In [69]:
#for fetching the toki from Training set and feature look up from old and it might have actions too .
def tokisLookUp(tokisID, oldTokisList, action=None):
    if action == None:
        filteredTokis = oldTokisList.loc[oldTokisList['referenceId'] == tokisID]
    else:
        filteredTokis = oldTokisList.loc[(oldTokisList['referenceId'] == tokisID) & (oldTokisList['action'] == action)]
        
    return filteredTokis
    
    

In [85]:
#for getting the association availble in association rule file from the tokis we looked up.
#future implementation should have the logic to bifurcate as per action
#ConfDict is for minimum constraints on confidence,support and lift
#workout the continuous nature of some of AR
def associationParser(filteredTokis, ARDataframe, ConfDict):
    #convert to association rules from csvpandas to dictionary and look for them in filteredTokis
    conf = float(ConfDict['confidence'])
    supp = float(ConfDict['support'])
    lift = float(ConfDict['lift'])
    configFullfillingAR = ARDataframe.loc[(ARDataframe['confidence'] > conf) & (ARDataframe['support'] > supp) & (ARDataframe['lift'] > lift)]
    #print(configFullfillingAR)
    for _,row in ARDataframe.iterrows():
        ruleLookup = dict(ast.literal_eval(row['lhs']))
        ruleLookup.update(dict(ast.literal_eval(row['rhs'])))
        #here seems to the problem check it
        recommendedTokis = filteredTokis.loc[filteredTokis[list(ruleLookup.keys())].isin(list(ruleLookup.values())).all(axis=1)]
    return recommendedTokis
    

In [84]:
#wrapper function for toki processing
def tokisProcessor(newTokisInput, jsonFile, oldTokisFilename, ARDataframe):
    try:
        with open(jsonFile) as f:
            confDict = json.load(f)
    except Exception as e:
        confDict = {'confidence':0,'support':0,'lift':0}
    try:
        oldTokisList = pd.read_csv(oldTokisFilename, sep = ',', header='infer', warn_bad_lines=True)

    except Exception as e:
        print('Old Tokis file not available or not in csv '+e)
    for _, row in newTokisInput.iterrows():
        tokisID = str(row['referenceId'])
        #calls to tokisLookup for each Toki
        lookupResult = tokisLookUp(tokisID,oldTokisList)
        
         #calls to AssociationParser for each Toki
        recommendations = associationParser(lookupResult, ARDataframe, confDict)
        
    return recommendations
       


In [64]:
#for the predicting the output for tokis, This will get changed when response is to be used they should be bifurcated as per action and should be in json format
#recommendation from tokiprocessor are in pandas dataframe and they needed to be converted to json
def outputHandler(recommendations):
    output = recommendations.to_json(orient='records')
    return output
    

In [86]:
#setUp
ARFrame = csvToPandasParser('AssociationRules.csv')
inputTokis = inputHandler('newTokis.csv')

#processing
recomm = tokisProcessor(inputTokis,'config.json','OldTokis.csv', ARFrame)
#printing for checking
print(outputHandler(recomm))

                            referenceId   tokiType tokiSubType      debtType  \
0  22aea424-3826-4be9-b99a-1c92ea78ae3b  Financing         APF  Non-Recourse   

  interestCollectionType financingType  \
0                PrePaid       Invoice   

                                 descriptions  tokiDate Country  Latittude  \
0  High Value 'TOKI' description number three  20190205   China   39.14222   

                   ...                    APR  industryType  goods DRA_score  \
0                  ...                   11.5        Energy  Sugar       118   

   DRA_code amount_ccy amount_value  amount_fundingPercentage  \
0       Low        GBP  18723767.64                     85.11   

                     supplierBusinessId                      buyersBusinessId  
0  1fc4051b-3a4d-4434-8383-5fed673c4fc3  ea0c3ed1-f32c-41ae-9d2d-3e3a0f145465  

[1 rows x 21 columns]
--------------
                            referenceId   tokiType tokiSubType      debtType  \
0  22aea424-3826-4be9-b99a-

In [53]:
ARFrame.head()

,actions,lhs,rhs,confidence,support,lift
0,Recomonded,"{'DRA_code':'low', 'industryType':'Energy', 'g...",{'Country':'China'},0.7,0.5,0.8
1,Interested,"{'Country':'China', 'interestCollectionType':'...",{'industryType':'Energy'},0.8,0.6,0.3


In [55]:
inputTokis.head()

,referenceId
0,22aea424-3826-4be9-b99a-1c92ea78ae3b
1,e487be97-0659-4545-a305-02ac7a66e748
2,bdcc05ee-ebb6-4162-9641-c19077a5fdc9


In [66]:
recomm.head()

,referenceId,tokiType,tokiSubType,debtType,interestCollectionType,financingType,descriptions,tokiDate,Country,Latittude,...,APR,industryType,goods,DRA_score,DRA_code,amount_ccy,amount_value,amount_fundingPercentage,supplierBusinessId,buyersBusinessId
